In [1]:
import geemap
import pandas as pd
import numpy as np
import pdb
from IPython.display import display
import ee
import os

In [2]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

#ee.Initialize()

0.34.1


In [3]:
# Define the region of interest for Georgia and Iowa
iowa = geemap.shp_to_ee('F:/US states/iowa.shp')

In [4]:
dem = ee.ImageCollection('USGS/3DEP/1m') \
    .filterBounds(iowa) \
    .mosaic()

dem_vis = {
  'min': 0,
  'max': 3000,
  'palette': [
    '3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13',
    'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', '0602ff', '235cb1',
    '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f'
  ]
}

hillshade_iowa = ee.Terrain.hillshade(dem.select('elevation'))

In [15]:
#Add date to image
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

#2015 imagery vs. 2010 BMP data? This is the earliest I could get for SAR, though
s1_col = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(iowa) \
    .filterDate('2015-01-01', '2015-12-31') \
    .filter(
        #Filter to get images with VV and VH dual polarization.
        ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')
    ) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(
        # Filter to get images collected in interferometric wide swath mode.
        ee.Filter.eq('instrumentMode', 'IW')
    )

s1_asc = s1_col.filter(
    ee.Filter.eq('orbitProperties_pass', 'ASCENDING')
)

s1_dsc = s1_col.filter(
    ee.Filter.eq('orbitProperties_pass', 'DESCENDING')
)

iowa_mosaic_2015 = s1_asc.mosaic()

#     .map(addDate) \
#     .mosaic() \
#     .addBands(dem.select('elevation'))

In [18]:
Map = geemap.Map(center=[40.6, -94], zoom=12)

pointx = ee.Geometry.Point(-94, 40.6)

Map.addLayer(iowa_mosaic_2015,{'min': -20, 'max': 10},'SAR')
Map.addLayer(pointx,{},'samplepoint')

# Map

In [13]:
# NLCD: create farmland buffer. Only data available is 2021 and 2019.
nlcd_iowa = ee.ImageCollection("USGS/NLCD_RELEASES/2021_REL/NLCD") \
    .filterBounds(iowa) \
    .filterDate('2001-01-01', '2021-12-31') \
    .select('landcover') \
    .mosaic() \
    .clip(iowa)

In [19]:
#Extract values from features
#Note: return feature.limit(5000) added to get the function to work for this large dataset - max per polygon
##UPDATE: FEATURE LIMIT SAMPLE SIZE must be BALANCED


# Function to extract values from features within buffer polygons
def dam_rasterExtraction_within_buffer(image, buffer_polygon):
    dam_clip_within_buffer = dam_clip.filterBounds(buffer_polygon.geometry())
    
    feature = image.sampleRegions(
        collection=dam_clip_within_buffer,
        scale=1, # Assuming NAIP imagery resolution
        geometries=True)
    
    return feature.limit(1000)



def terrace_rasterExtraction_within_buffer(image, buffer_polygon):
    terrace_clip_within_buffer = terrace_clip.filterBounds(buffer_polygon.geometry())
    feature = image.sampleRegions(
        collection=terrace_clip_within_buffer,
        scale=1, # Assuming NAIP imagery resolution
        geometries=True
    )
    
    elev_values = dem.sampleRegions(
        collection=terrace_clip_within_buffer,
        scale=1,
        geometries=True
    )

    return feature.limit(1000)



def basin_rasterExtraction_within_buffer(image, buffer_polygon):
    basin_clip_within_buffer = basin_clip.filterBounds(buffer_polygon.geometry())
    feature = image.sampleRegions(
        collection=basin_clip_within_buffer,
        scale=1, # Assuming NAIP imagery resolution
        geometries=True
    )
    
    elev_values = dem.sampleRegions(
        collection=basin_clip_within_buffer,
        scale=1,
        geometries=True
    )
    
    return feature.limit(1000)



def grassed_rasterExtraction_within_buffer(image, buffer_polygon):
    grassed_clip_within_buffer = grassed_clip.filterBounds(buffer_polygon.geometry())
    feature = image.sampleRegions(
        collection=grassed_clip_within_buffer,
        scale=1, # Assuming NAIP imagery resolution
        geometries=True
    )
    
    return feature.limit(1000)



def contour_rasterExtraction_within_buffer(image, buffer_polygon):
    contour_clip_within_buffer = contour_clip.filterBounds(buffer_polygon.geometry())
    feature = image.sampleRegions(
        collection=contour_clip_within_buffer,
        scale=1, # Assuming NAIP imagery resolution
        geometries=True
    )
    
    return feature.limit(1000)



def strip_rasterExtraction_within_buffer(image, buffer_polygon):
    strip_clip_within_buffer = strip_clip.filterBounds(buffer_polygon.geometry())
    feature = image.sampleRegions(
        collection=strip_clip_within_buffer,
        scale=1, # Assuming NAIP imagery resolution
        geometries=True
    )
    
    elev_values = dem.sampleRegions(
        collection=strip_clip_within_buffer,
        scale=1,
        geometries=True
    )
    
    return feature.limit(1000)



#Dummy farm variable using NLCD - NOTE THE DIFFERENT FORMAT IN HOW THE FUNCTION WORKS
def farm_rasterExtraction_within_buffer(image, region):
    feature = image.sampleRegions(
        collection=region,
        scale=1, # Assuming NAIP imagery resolution - NLCD is 30 m though
        geometries=True
    )
    
    return feature.limit(5000)

In [21]:
#Random buffer zones - 100 of them. NOTE: in earlier version, _buffer was used instead of _clip

# buffer_points = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_areas/buffer.shp')
buffer_points = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_areas/buffer2.shp') #no overlaps

# contour_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/contour_buffer.shp')
# grassed_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/grassed_buffer.shp')
# dam_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/dam_buffer.shp')
# strip_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/strip_buffer.shp')
# terrace_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/terrace_buffer.shp')
# basin_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips/basin_buffer.shp')

#Switched to bufferclips2 folder - no overlaps, 100 polygons
contour_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/contour_buffer.shp')
grassed_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/grassed_buffer.shp')
dam_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/dam_buffer.shp')
strip_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/strip_buffer.shp')
terrace_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/terrace_buffer.shp')
basin_clip = geemap.shp_to_ee('F:/Iowa BMP/Iowa clipped shapefiles/buffer_clips2/basin_buffer.shp')

In [22]:
#Create a mask for BMP clips:

def maskInside(image, geometry):
    mask = ee.Image.constant(1).clip(geometry).mask().Not()
    return image.updateMask(mask)

def maskFarmland(image):
    mask = image.eq(71).Or(image.eq(81)).Or(image.eq(82))
    return image.updateMask(mask)


# Combine all feature geometries into a single multipolygon
all_features = contour_clip.merge(grassed_clip).merge(dam_clip).merge(strip_clip).merge(terrace_clip).merge(basin_clip)
all_multipolygon = all_features.geometry()

# Apply the  mask to the NLCD image
# nlcd_filtered = maskInside(nlcd_mask, all_multipolygon)

nlcd_filtered = maskInside(maskFarmland(nlcd_iowa), all_multipolygon)

In [ ]:
# Create a map

iowa_masked = iowa_mosaic_2015.updateMask(nlcd_filtered)

# Map.addLayer(nlcd_filtered, {}, 'Filtered NLCD')

# Map.addLayer(dem, dem_vis, 'elevation')
# Map.addLayer(hillshade_iowa)

Map.addLayer(iowa_masked, {}, 'Masked')

Map.addLayer(buffer_points, {}, 'buffer points')

# Map

In [ ]:
#Convert buffer polygon shapefile into list of all polygon features
buffer_list = buffer_points.toList(buffer_points.size())

In [ ]:
#Dam
# Initialize an empty list to store the results
result_list = []

# Iterate over each buffer polygon and extract raster values within each buffer
for i in range(buffer_list.size().getInfo()):
    try:
        buffer_polygon = ee.Feature(buffer_list.get(i))
        result = geemap.ee_to_pandas(dam_rasterExtraction_within_buffer(iowa_mosaic_2010, buffer_polygon))
        result_list.append(result)
    except Exception:
        continue

# Merge the results into a single dataframe
dam_mosaic = pd.concat(result_list)

In [ ]:
#Terrace
# Initialize an empty list to store the results
result_list = []

# Iterate over each buffer polygon and extract raster values within each buffer
for i in range(buffer_list.size().getInfo()):
    try:
        buffer_polygon = ee.Feature(buffer_list.get(i))
        result = geemap.ee_to_pandas(terrace_rasterExtraction_within_buffer(iowa_mosaic_2010, buffer_polygon))
        result_list.append(result)
    except Exception:
        continue

# Merge the results into a single dataframe
terrace_mosaic = pd.concat(result_list)


In [ ]:
#Basin
# Initialize an empty list to store the results
result_list = []

# Iterate over each buffer polygon and extract raster values within each buffer
for i in range(buffer_list.size().getInfo()):
    try:
        buffer_polygon = ee.Feature(buffer_list.get(i))
        result = geemap.ee_to_pandas(basin_rasterExtraction_within_buffer(iowa_mosaic_2010, buffer_polygon))
        result_list.append(result)
    except Exception:
        continue

# Merge the results into a single dataframe
basin_mosaic = pd.concat(result_list)


In [ ]:
#Grassed
# Initialize an empty list to store the results
result_list = []

# Iterate over each buffer polygon and extract raster values within each buffer
for i in range(buffer_list.size().getInfo()):
    try:
        buffer_polygon = ee.Feature(buffer_list.get(i))
        result = geemap.ee_to_pandas(grassed_rasterExtraction_within_buffer(iowa_mosaic_2010, buffer_polygon))
        result_list.append(result)
    except Exception:
        continue

# Merge the results into a single dataframe
grassed_mosaic = pd.concat(result_list)

In [ ]:
#Contour
# Initialize an empty list to store the results
result_list = []

# Iterate over each buffer polygon and extract raster values within each buffer
for i in range(buffer_list.size().getInfo()):
    try:
        buffer_polygon = ee.Feature(buffer_list.get(i))
        result = geemap.ee_to_pandas(contour_rasterExtraction_within_buffer(iowa_mosaic_2010, buffer_polygon))
        result_list.append(result)
    except Exception:
        continue

# Merge the results into a single dataframe
contour_mosaic = pd.concat(result_list)

In [ ]:
#Strip
# Initialize an empty list to store the results
result_list = []

# Iterate over each buffer polygon and extract raster values within each buffer
for i in range(buffer_list.size().getInfo()):
    try:
        buffer_polygon = ee.Feature(buffer_list.get(i))
        result = geemap.ee_to_pandas(strip_rasterExtraction_within_buffer(iowa_mosaic_2010, buffer_polygon))
        result_list.append(result)
    except Exception:
        continue

# Merge the results into a single dataframe
strip_mosaic = pd.concat(result_list)

In [ ]:
#NON-BMP FARMLAND - NOTE THE DIFFERENT SYNTAX

# Initialize an empty list to store the results
iowa_masked = iowa_mosaic_2010.updateMask(nlcd_filtered)

result_list = []

# Iterate over each buffer polygon and extract raster values within each buffer
for i in range(buffer_list.size().getInfo()):
    try:
        region = ee.Feature(buffer_list.get(i))
        result = geemap.ee_to_pandas(farm_rasterExtraction_within_buffer(iowa_masked, region))
        result_list.append(result)
    except Exception:
        continue

# Merge the results into a single dataframe
farmland_mosaic = pd.concat(result_list)

farmland_mosaic['PRACTICE'] = 'Farmland'

farmland_mosaic

In [ ]:
#NON-BMP FARMLAND - NOTE THE DIFFERENT SYNTAX

# Initialize an empty list to store the results
iowa_masked = iowa_mosaic_2010.updateMask(nlcd_filtered)

result_list = []

# Iterate over each buffer polygon and extract raster values within each buffer
for i in range(buffer_list.size().getInfo()):
    try:
        region = ee.Feature(buffer_list.get(i))
        result = geemap.ee_to_pandas(farm_rasterExtraction_within_buffer(iowa_masked, region))
        result_list.append(result)
    except Exception:
        continue

# Merge the results into a single dataframe
farmland_mosaic = pd.concat(result_list)

farmland_mosaic['PRACTICE'] = 'Farmland'

farmland_mosaic

In [ ]:
ulti_log

# for col in ulti_log.columns:
#     print(col)